In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import TavilySearchResults
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os
# 🔐 Load API keys
load_dotenv(".env")
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Simple QA tool (LLMChain)
qa_prompt = PromptTemplate.from_template("Answer clearly: {question}")
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
qa_tool = Tool(
    name="Simple_QA",
    func=qa_chain.run,
    description="Answer the question without hallucination"
)

# Web search tool (Tavily)
search_tool = Tool(
    name="Web_Search",
    func=TavilySearchResults(max_results=3).run,
    description="Search the internet for current info"
)

tools = [qa_tool, search_tool]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [25]:
#1) Zero-Shot ReAct (simple + reliable default)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")



> Entering new AgentExecutor chain...
I should search the internet for information on LangChain and its creator.
Action: Web_Search
Action Input: "LangChain summary" 
Observation: [{'title': 'What Is LangChain? Examples and definition - Google Cloud', 'url': 'https://cloud.google.com/use-cases/langchain', 'content': '# What is LangChain?\n\nLangChain is a programming language platform that lets developers construct and connect models to access, transform, and share data seamlessly. It offers a powerful and versatile environment for model development, enabling the manipulation of data structures, model chaining, and the incorporation of external agents like LaMDA. [...] LangChain operates on the principle of modularity, decomposing language-based AI systems into reusable components. These components, known as "chains," encapsulate specific functionalities, such as data retrieval, model interaction, and memory management. By assembling these chains in various configurations, developers

In [10]:
#2) Conversational ReAct (with memory; non-chat model baseline)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Simple QA
Action Input: Summarize LangChain in 2 lines
Observation: LangChain is a decentralized platform that connects language service providers with clients, offering secure and efficient language translation services. It utilizes blockchain technology to ensure transparency, trust, and fair compensation for all parties involved.
Do I need to use a tool? Yes
Action: Simple QA
Action Input: Who created LangChain?
Observation: LangChain was created by a team of developers and language enthusiasts led by its founder, Dr. Alex Waibel.
Do I need to use a tool? No
AI: LangChain is a decentralized platform that connects language service providers with clients, offering secure and efficient language translation services. It was created by a team of developers and language enthusiasts led by its founder, Dr. Alex Waibel.

> Finished chain.


In [29]:
#3) Chat Conversational ReAct (recommended for GPT-3.5/4 chat)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)
result = agent.run("Plan a 3-step study path for LangChain.")
#result = agent.run("Summarize LangChain in 2 lines, then tell me who created it.")




> Entering new AgentExecutor chain...
```json
{
    "action": "Simple_QA",
    "action_input": "Plan a 3-step study path for LangChain"
}
```
Observation: Step 1: Begin by familiarizing yourself with the basics of LangChain, including its purpose, features, and how it works. This can be done by reading the whitepaper, watching introductory videos, and exploring the official website.

Step 2: Dive deeper into LangChain by studying its technical aspects, such as its underlying technology, consensus mechanism, and security features. This can be achieved by reading technical documentation, participating in online forums and discussions, and attending webinars or workshops.

Step 3: Apply your knowledge of LangChain by experimenting with the platform, such as setting up a test environment, creating smart contracts, and interacting with the community. This hands-on experience will help solidify your understanding of LangChain and prepare you for more advanced studies or projects in the fut

In [14]:
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel

# ✅ Define the input schema using Pydantic
class TitleInput(BaseModel):
    topic: str
    tone: str = "concise"

# ✅ Define the tool function
def title_tool_fn(topic: str, tone: str = "concise") -> str:
    return f"{tone.title()} Title: {topic} in Practice"

# ✅ Create the structured tool
title_tool = StructuredTool.from_function(
    name="TitleMaker",
    func=title_tool_fn,
    description="Generate a title given a topic and an optional tone.",
    args_schema=TitleInput
)

# ✅ Initialize the agent
agent = initialize_agent(
    tools=[title_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Run the agent
result = agent.run("Make a friendly title about LangGraph tutorials.")
print("✅ Result:", result)




> Entering new AgentExecutor chain...
Thought: I can use the TitleMaker tool to generate a friendly title about LangGraph tutorials.
Action:
```
{
  "action": "TitleMaker",
  "action_input": {
    "topic": "LangGraph tutorials",
    "tone": "friendly"
  }
}
```
Observation: Friendly Title: LangGraph tutorials in Practice
I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "LangGraph tutorials in Practice"
}
```

> Finished chain.
✅ Result: LangGraph tutorials in Practice


In [17]:
#5)OpenAI Functions (function-calling schema)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
result = agent.run("Search the web for LangGraph docs and give me 3 bullets.")




> Entering new AgentExecutor chain...

Invoking: `Web_Search` with `LangGraph documentation`


[{'title': 'Reference - GitHub Pages', 'url': 'https://langchain-ai.github.io/langgraph/reference/', 'content': "Welcome to the LangGraph reference docs! These pages detail the core interfaces you will use when building with LangGraph. Each section covers a different part of the ecosystem.\n\nTip\n\nIf you are just getting started, see LangGraph basics for an introduction to the main concepts and usage patterns.\n\nLangGraph¶\n--------------------------------------------------------------------------------------------\n\nThe core APIs for the LangGraph open source library. [...] Published Time: Mon, 04 Aug 2025 19:34:46 GMT\n\nReference\n\n===============\n- [x] - [x] \n\nSkip to content\n\nLangGraph Platform docs have moved! Find the LangGraph Platform docs at the new LangChain Docs site. \n\nImage 1: logoImage 2: logo\n\n LangGraph \n\n Reference \n\n\n\n Initializing search \n\nGitHub  0

In [19]:
#6) OpenAI Multi-Functions (can call multiple tools in one step)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True
)
result = agent.run("Answer briefly from QA; also check the web for any updates.")




> Entering new AgentExecutor chain...

Invoking: `Simple_QA` with `{'tool_input': 'Answer briefly from QA'}`


What is the capital of France?
- Paris
Invoking: `Web_Search` with `{'tool_input': 'Check for any updates'}`


[{'title': 'CNN: Breaking News, Latest News and Videos', 'url': 'https://www.cnn.com/', 'content': 'Image 1: 20250731-Epstein-timeline-2b.jpg • For Subscribers For Subscribers Photo Illustration by Alberto Mier/CNN/Getty Images• For Subscribers For Subscribers For Subscribers Donald Trump and Jeffrey Epstein’s relationship: A visual timeline Image 15: image.png Molly Flores/CNN UnderscoredCNN Underscored Gear up for back-to-school season with these 73 editor-approved deals Aug 5, 2025 Image 36: Apple Intelligence Product - 1160x653 (1).jpg James Harris/CNN UnderscoredApple will invest $100 billion to make US iPhone parts as Trump says a big chip tariff is coming Aug 6, 2025 Image 56: Getty_Villa_1.jpg Gabe Ramirez, CNNVideo In a city prone to natural disasters, this

In [28]:
#7) Self-Ask with Search (clarifies, then searches)

search_tool = Tool(
    name="Intermediate Answer",  # This name is REQUIRED
    func=TavilySearchResults(max_results=1).run,
    description="Use this tool to search for missing facts"
)

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    handle_parsing_errors=True
)
result = agent.run("When was the first LangChain release and who founded it?")




> Entering new AgentExecutor chain...
Yes. 
Follow up: When was the first LangChain release?
Intermediate answer: [{'title': 'LangChain', 'url': 'https://en.wikipedia.org/wiki/LangChain', 'content': '| Connery | No | Yes | Commercial | API actions | https://python.langchain.com/docs/integrations/tools/connery | | Dall-E Image Generator | No | Yes | Proprietary | Text-to-image generation | https://python.langchain.com/docs/integrations/tools/dalle_image_generator | | DuckDuckGo Search | No | No | Open source | Privacy-focused search | https://python.langchain.com/docs/integrations/tools/ddg | | Google Scholar | Yes | Yes | Proprietary | Scholarly article search | https://python.langchain.com/docs/integrations/tools/google_scholar | | HuggingFace Hub | No | No | Open source | Hugging Face models, datasets | https://python.langchain.com/docs/integrations/tools/huggingface_tools | | SerpAPI | No | Yes | Commercial | Search engine results page scraping | https://python.langchain.com/docs/

KeyboardInterrupt: 

![Model Diagram](agenttypes1.png)

![Model Diagram](agenttype2.png)

In [ ]:
Deep Dive

![Model Diagram](pro1.png)

![Model Diagram](pro2.png)

![Model Diagram](pro3.png)